<a href="https://colab.research.google.com/github/darksun373/sf_data_science/blob/main/sf_pandas_04_06_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с Pandas

Булыгин Олег  
 
* [LinkedIn](linkedin.com/in/obulygin)  
* [Мой ТГ канал по Python](https://t.me/solidtalk)

## План
В рамках воркфлоу работы с открытым датафреймом по реальным транзакциям британского онлайн-магазина подарков. Реализуем набор аналитических действий и решим задачи:
- посчитаем статистику по странам;
- проанализируем выручку;
- изучим статистику по клиентам;
- проанализируем динамику среднего чека;
- просегментируем товары по стоимости;
- сделаем когортный анализ и анализ удержания клиентов (уже непросто!).

Пройдя по этим шагам повторим:
- базовую работу с датами;
- преобразование типов;
- сортировку;
- группировку и сводные таблицы;
- объединение датафреймов.

In [ ]:
# импортируем pandas
import pandas as pd

### Открытие и знакомство с данными

In [ ]:
# читаем данные из csv, который находится в zip-файле на github
url = 'https://github.com/obulygin/SkillFactory/blob/main/ecommerce-data.zip?raw=true'
df = pd.read_csv(url, encoding='ISO-8859-1', compression='zip')

In [ ]:
# смотрим первые 5 строк таблицы
display(df.head())
# смотрим последние 5 строк таблицы
display(df.tail())

In [ ]:
# рассчитываем базовые статистики по каждому числовому признаку
df.describe() 

### Избавимся от отрицательных значений

In [ ]:
# оставляем данные только с положительными ценами на товар и количеством
df = df[(df['UnitPrice'] > 0) & (df['Quantity'] > 0)]
df.describe()

### Избавимся от дубликатов в данных

In [ ]:
# посмотрим, сколько у нас повторов
print(df.duplicated().sum())

In [ ]:
# сколько всего строк?
print(df.shape)

In [ ]:
# удалим дубли и проверим
df.drop_duplicates(inplace=True)
print(df.shape)

### Поправим типы данных

In [ ]:
df.info()

In [ ]:
# почему ошибка?
df['CustomerID'] = df['CustomerID'].astype('int64')

In [ ]:
# Nullable datatypes: https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
df['CustomerID'] = df['CustomerID'].astype('Int64')

In [ ]:
# переведем даты из строкового формата в datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df.info()

### Изучаем статистику по странам

In [ ]:
# считаем сколько транзакций было по странам при помощи value_counts
df['Country'].value_counts()

In [ ]:
# считаем сколько уникальных клиентов в странах
countries = df.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False)
countries

#### Практика. Определяем топ-5 лояльных клиентов по количеству покупок

### Создадим временные признакми

In [ ]:
# добавим месяц покупки в новый столбец при помощи dt.strftime('%Y - %m')
df['InvoiceMonth'] = df['InvoiceDate'].dt.strftime('%Y-%m') 

In [ ]:
# уникальные покупатели по месяцам
df.groupby('InvoiceMonth')['CustomerID'].nunique()

In [ ]:
# зафиксируем дни недели
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek + 1

In [ ]:
# посчитаем количество заказов по дням недели
df.groupby('DayOfWeek')['InvoiceNo'].nunique()


#### Практика. Изучим динамику количества покупок по часам

### Посчитаем выручку

In [ ]:
# создадим новый столбец
df['Revenue'] = df['Quantity'] * df['UnitPrice']

df.head() 

In [ ]:
# выручка по месяцам
df.groupby('InvoiceMonth')['Revenue'].sum()

#### Практика. Найдите 3 страны, которые принесли максимальную выручку после UK

### Сегменты товаров по стоимости

In [ ]:
# напишем функцию для сегментации товаров
def price_segment(row):
    if row < 5:
        return 'cheap' 
    elif 5 <= row < 25:
        return 'lower-middle'
    elif 25 <= row < 100:
        return 'upper-middle'
    elif row >= 100:
        return 'expensive'
 

df['PriceSegment'] = df['UnitPrice'].apply(price_segment)
df['PriceSegment'].value_counts()

#### Практика. Определить, какая из категорий товаров принесла больше всего выручки

### Как меняется средний чек со временем?

In [ ]:
df.groupby('InvoiceMonth')['Revenue'].sum().divide(df.groupby('InvoiceMonth')['InvoiceNo'].nunique())

### Построим сводные таблицы

In [ ]:
# сводная таблица по выручки в определенной стране в разрезе пользователей и месяцев
country = 'France'
table = pd.pivot_table(
    df[df['Country']==country], 
    index='CustomerID', 
    columns='InvoiceMonth', 
    aggfunc='sum', 
    values='Revenue', 
    fill_value=0)


# подсветить максимальные значения по каждому месяцу
# table
table.style.highlight_max()
# table.style.background_gradient(cmap='PuBu')

#### Практика. Построить сводную таблицу c кол-вом заказов по странам и месяцам


In [ ]:
# изучим сводную статистику по покупателям из произвольной страны (суммарная и средняя выручка)
products = df[df['Country']==country].groupby(['CustomerID', 'InvoiceNo'], as_index=False)['Revenue'].sum()
display(products)
products = products.groupby('CustomerID').agg({'InvoiceNo': ['count'], 'Revenue': ['sum','mean']})
display(products)

In [ ]:
# объединяем статистику по месяцам и статистику по выручки и количеству заказов в разрезе пользователей
table_plus = pd.merge(table, products, on='CustomerID') 
table_plus.style.highlight_max()
# table_plus.style.background_gradient(cmap='PuBu')

### Когортный анализ

Идея когортного анализа состоит в том, чтобы резделить пользоватей на группы по определенным признакам, и отслеживать поведение этих групп во времени. Обычно группы пользователей (когорты) выделяют на основе недели (месяца), когда пользователи совершили первую покупку.

Выделив такие группы пользователей (когорты), вы следите за ними в течение времени и измеряете ключевые метрики для каждой отдельной когорты.

In [ ]:
# определяем месяц первой покупки для каждого покупателя 
cohorts = df.groupby('CustomerID', as_index=False)['InvoiceMonth'].min()
cohorts.rename(columns = {'InvoiceMonth':'Cohort'}, inplace=True)
cohorts.head()

In [ ]:
# добавим когорты к основным данным

# исключаем пропуски по пользователям для корректного объединения
df_merged = pd.merge(df.dropna(), cohorts, how='left', on='CustomerID')
df_merged.head()

In [ ]:
# считаем доходы по когортам
df_merged.groupby('Cohort')['Revenue'].sum()

### Анализ retention

Customer retention (удержание клиентов) — это способность компании удерживать своих потребителей в течение определенного периода времени.

In [ ]:
import numpy as np

# пишем функцию которая добавляет столбец с временем жизни каждой когорты
def cohort_period(df):
  df['CohortLT'] = np.arange(len(df))
  return df

# считаем сколько уникальных клиентов у нас в каждой когорте
cohorts = df_merged.groupby(['Cohort', 'InvoiceMonth'])[['CustomerID']].nunique()
cohorts = cohorts.groupby(level=0).apply(cohort_period)
cohorts.rename({'CustomerID': 'TotalUsers'}, inplace=True)
cohorts

In [ ]:
# переведем когорты в индекс, а дату чека в столбец
cohorts.reset_index(inplace=True)
cohorts.set_index(['Cohort', 'CohortLT'], inplace=True) 
cohorts = cohorts.rename(columns={'CustomerID': 'TotalUsers'})
cohorts

In [ ]:
# считаем ретеншн пользователей в каждой когорте

# определим количество покупок в первый месяц когорты (100%)
cohort_group_size = cohorts['TotalUsers'].groupby(level=0).first()
cohort_group_size


In [ ]:
# узнаем какой процент покупок этой когортой совершался в последующие месяца
user_retention = cohorts['TotalUsers'].unstack(0).divide(cohort_group_size, axis=1)
user_retention

**Спасибо за внимание буду рад ответить на ваши вопросы!**  
Ссылка на форму ОС:
https://forms.gle/y8xaFwJqtbFSjUeG8